In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import time
import jieba

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process01', sep=',', header=None, names=['from', 'to', 'date', 'content', 'label'])
df.head(10)

,from,to,date,content,label
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,Sun 14 Aug 2005 10:17:57 +0800,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1.0
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,Sun 14 Aug 2005 10:19:02 +0800,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1.0
4,mei <mei@dghhkjk.com>,tang@ccert.edu.cn,Sun 14 Aug 2005 10:21:22 +0800,这是一封HTML格式信件！---------------------------------...,1.0
5,"ke@163.com"" <chunyang-sz@163.com>",yuan@ccert.edu.cn,Sun 14 Aug 2005 10:22:10 +0800,TO：贵公司经理、财务您好！深圳市春洋贸易有限公司（东莞分公司）我司本着互惠互利的优势和良好...,1.0
6,hong <hong@jdl.ac.cn>,yu@ccert.edu.cn,Sun 14 Aug 2005 10:23:37 +0800,那他为什么不愿意起诉，既然这样了！起诉后也有充分的理由！MM莫不是还生活在电影中，个人认为这...,0.0
7,=?GB2312?B?wbrPyMn6?= <jiang@tom.com>,li@ccert.edu.cn,Sun 14 Aug 2005 10:26:36 +0800,尊敬的负责人（经理／财务）：您好！我是深圳伟仕嘉贸易有公司：兴办贸易、物资供销，实力雄厚；有...,1.0
8,han <han@davidchans.com>,lai@ccert.edu.cn,Sun 14 Aug 2005 10:27:40 +0800,您好 以下是特别为阁下发的香港信息(图片、景点、BBS等) 不知道阁下是否喜...希望没有打...,1.0
9,hou <hou@jdl.ac.cn>,li@ccert.edu.cn,Sun 14 Aug 2005 10:31:20 +0800,我觉得，负债不要紧，最重要的是能负得起这个责任来，欠了那么多钱，至少对当初拿出爱心来的网友们...,0.0


In [5]:
# 2. 特征工程1 => 提取发件人和收件人的邮件服务器地址
def extract_email_server_address(str1):
    it = re.findall(r"@([A-Za-z0-9]*\.[A-Za-z0-9\.]+)", str(str1))
    result = ''
    if len(it) > 0:
        result = it[0]
    if not result:
        result = 'unknown'
    return result

df['to_address'] = pd.Series(map(lambda str: extract_email_server_address(str), df['to']))
df['from_address'] = pd.Series(map(lambda str: extract_email_server_address(str), df['from']))

df.head(4)

,from,to,date,content,label,to_address,from_address
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0,ccert.edu.cn,unknown
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0,ccert.edu.cn,jdl.ac.cn
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,Sun 14 Aug 2005 10:17:57 +0800,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1.0,ccert.edu.cn,163.con
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,Sun 14 Aug 2005 10:19:02 +0800,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1.0,ccert.edu.cn,12.com


In [6]:
# 2. 特征工程1 => 查看邮件服务器的数量
print("========to address=======================")
print(df.to_address.value_counts().head(5))
print("总邮件接收服务器类别数量为:" + str(df.to_address.unique().shape))
print("========from address=======================")
print(df.from_address.value_counts().head(5))
print("总邮件发送服务器类别数量为:" + str(df.from_address.unique().shape))
from_address_df = df.from_address.value_counts().to_frame()
len_less_10_from_address_count = from_address_df[from_address_df.from_address <= 10].shape
print("发送邮件数量小于10封的服务器数量为:" + str(len_less_10_from_address_count))

========to address=======================
ccert.edu.cn    64407
unknown           193
yahoo.com.cn        8
163.net             3
quanso.com          2
Name: to_address, dtype: int64
总邮件接收服务器类别数量为:(12,)
========from address=======================
163.com                  7500
mail.tsinghua.edu.cn     6498
126.com                  5822
tom.com                  4075
mails.tsinghua.edu.cn    3205
Name: from_address, dtype: int64
总邮件发送服务器类别数量为:(3567,)
发送邮件数量小于10封的服务器数量为:(3202, 1)


In [7]:
np.unique(list(map(lambda t: len(str(t).strip()), df['date'])))

array([ 3,  7, 16, 19, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 45, 46, 57, 58, 61, 62])

In [8]:
np.unique(list(filter(lambda t: len(str(t).strip()) == 24, df['date'])))

array(['Fri 16 Sep 2005 04:35:34', 'Fri 16 Sep 2005 10:20:18',
       'Fri 16 Sep 2005 11:16:04', 'Fri 16 Sep 2005 17:57:28',
       'Fri 16 Sep 2005 21:03:05', 'Fri 19 Aug 2005 10:16:38',
       'Fri 23 Sep 2005 17:46:34', 'Fri 26 Aug 2005 22:44:48',
       'Fri 30 Sep 2005 00:50:39', 'Fri 30 Sep 2005 16:47:54',
       'Mon 15 Aug 2005 07:04:08', 'Mon 19 Sep 2005 13:08:22',
       'Mon 26 Sep 2005 17:06:10', 'Sat 10 Sep 2005 09:51:18',
       'Sat 10 Sep 2005 16:53:55', 'Sat 20 Aug 2005 12:08:37',
       'Sat 27 Aug 2005 04:25:02', 'Sun 18 Sep 2005 04:03:31',
       'Sun 18 Sep 2005 17:13:12', 'Sun 18 Sep 2005 17:50:39',
       'Sun 28 Aug 2005 05:19:22', 'Sun 28 Aug 2005 05:21:34',
       'Thu 15 Sep 2005 07:36:45', 'Thu 15 Sep 2005 16:18:40',
       'Thu 15 Sep 2005 16:34:01', 'Thu 15 Sep 2005 22:57:27',
       'Thu 22 Sep 2005 14:51:30', 'Thu 29 Sep 2005 10:37:31',
       'Thu 29 Sep 2005 15:39:48', 'Tue 20 Sep 2005 07:20:03',
       'Tue 23 Aug 2005 09:58:26', 'Tue 23 Aug 2005 10:

In [9]:
# 3. 特征工程2 => 时间提取
def extract_email_date(str1):
    if not isinstance(str1, str):
        str1 = str(str1)

    str_len = len(str1)
    week = ""
    hour = ""
    # 0表示上午[8,12]，1表示下午[13,18],2表示晚上[19,23],3表示凌晨[0,7]
    time_quantum = ""

    if str_len < 10:
        # unknown
        week = "unknown"
        hour = "unknown"
        time_quantum = "unknown"
        pass
    elif str_len == 16:
        # 2005-9-2 上午10:55, 2005-9-2 上午11:04
        rex = r"(\d{2}):\d{2}"
        it = re.findall(rex, str1)
        if len(it) == 1:
            hour = it[0]
        else:
            hour = "unknown"
        week = "Fri"
        time_quantum = "0"
        pass
    elif str_len == 19:
        # Sep 23 2005 1:04 AM
        week = "Sep"
        hour = "01"
        time_quantum = "3"
        pass
    elif str_len == 21:
        # August 24 2005 5:00pm
        week = "Wed"
        hour = "17"
        time_quantum = "1"
        pass
    else:
        rex = r"([A-Za-z]+\d?[A-Za-z]*) .*?(\d{2}):\d{2}:\d{2}.*"
        it = re.findall(rex, str1)
        if len(it) == 1 and len(it[0]) == 2:
            week = it[0][0][-3:]
            hour = it[0][1]
            int_hour = int(hour)
            if int_hour < 8:
                time_quantum = "3"
            elif int_hour < 13:
                time_quantum = "0"
            elif int_hour < 19:
                time_quantum = "1"
            else:
                time_quantum = "2"
            pass
        else:
            week = "unknown"
            hour = "unknown"
            time_quantum = "unknown"

    week = week.lower()
    hour = hour.lower()
    time_quantum = time_quantum.lower()
    return (week, hour, time_quantum)

# 数据转换
date_time_extract_result = list(map(lambda st: extract_email_date(st), df['date']) )
df['date_week'] = pd.Series(map(lambda t: t[0], date_time_extract_result))
df['date_hour'] = pd.Series(map(lambda t: t[1], date_time_extract_result))
df['date_time_quantum'] = pd.Series(map(lambda t: t[2], date_time_extract_result))
df.head(4)

,from,to,date,content,label,to_address,from_address,date_week,date_hour,date_time_quantum
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0,ccert.edu.cn,unknown,tue,10,0
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0,ccert.edu.cn,jdl.ac.cn,sun,10,0
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,Sun 14 Aug 2005 10:17:57 +0800,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1.0,ccert.edu.cn,163.con,sun,10,0
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,Sun 14 Aug 2005 10:19:02 +0800,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1.0,ccert.edu.cn,12.com,sun,10,0


In [10]:
print("======星期属性字段的描述==========")
print(df.date_week.value_counts().head(3))
print(df[['date_week', 'label']].groupby(['date_week', 'label'])['label'].count())

======星期属性字段的描述==========
fri    10859
sat    10316
thu     9780
Name: date_week, dtype: int64
date_week  label
fri        0.0      3884
           1.0      6975
mon        0.0      2568
           1.0      5491
sat        0.0      3681
           1.0      6635
sep        0.0         1
sun        0.0      2785
           1.0      5724
thu        0.0      3330
           1.0      6450
tue        0.0      2733
           1.0      5399
unknown    1.0       553
wed        0.0      2784
           1.0      5626
Name: label, dtype: int64


In [11]:
print("======小时属性字段的描述==========")
print(df.date_hour.value_counts().head(3))
print(df[['date_hour', 'label']].groupby(['date_hour', 'label'])['label'].count())

======小时属性字段的描述==========
19    2835
16    2772
15    2750
Name: date_hour, dtype: int64
date_hour  label
00         0.0       904
           1.0      1716
01         0.0       925
           1.0      1791
02         0.0       868
           1.0      1736
03         0.0       839
           1.0      1682
04         0.0       824
           1.0      1771
05         0.0       822
           1.0      1791
06         0.0       758
           1.0      1748
07         0.0       863
           1.0      1775
08         0.0       801
           1.0      1732
09         0.0       896
           1.0      1795
10         0.0       874
           1.0      1847
11         0.0       889
           1.0      1779
12         0.0       936
           1.0      1740
13         0.0       909
           1.0      1712
14         0.0       945
           1.0      1757
15         0.0       979
           1.0      1771
16         0.0       988
           1.0      1784
17         0.0       940
           1.0     

In [12]:
print("======时间段属性字段的描述==========")
print(df.date_hour.value_counts().head(3))
print(df[['date_time_quantum', 'label']].groupby(['date_time_quantum', 'label'])['label'].count())

======时间段属性字段的描述==========
19    2835
16    2772
15    2750
Name: date_hour, dtype: int64
date_time_quantum  label
0                  0.0       4396
                   1.0       8893
1                  0.0       5756
                   1.0      10570
2                  0.0       4811
                   1.0       8827
3                  0.0       6803
                   1.0      14010
unknown            1.0        553
Name: label, dtype: int64


In [13]:
# 添加是否有时间
df['has_date'] = df.apply(lambda c: 0 if c['date_week'] == 'unknown' else 1, axis=1)
df.head(4)

,from,to,date,content,label,to_address,from_address,date_week,date_hour,date_time_quantum,has_date
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0,ccert.edu.cn,unknown,tue,10,0,1
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0,ccert.edu.cn,jdl.ac.cn,sun,10,0,1
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,Sun 14 Aug 2005 10:17:57 +0800,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1.0,ccert.edu.cn,163.con,sun,10,0,1
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,Sun 14 Aug 2005 10:19:02 +0800,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1.0,ccert.edu.cn,12.com,sun,10,0,1


In [14]:
## 将文本类型全部转换为str类型，然后进行分词操作
df['content'] = df['content'].astype('str')
# jieba添加分词字典 jieba.load_userdict("userdict.txt")
df['jieba_cut_content'] = list(map(lambda st: "  ".join(jieba.cut(st)), df['content']))
df.head(4)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ibf\AppData\Local\Temp\jieba.cache
Loading model cost 1.647 seconds.
Prefix dict has been built succesfully.


,from,to,date,content,label,to_address,from_address,date_week,date_hour,date_time_quantum,has_date,jieba_cut_content
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0,ccert.edu.cn,unknown,tue,10,0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0,ccert.edu.cn,jdl.ac.cn,sun,10,0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...
2,=?GB2312?B?1cW6o8TP?= <jian@163.con>,xing@ccert.edu.cn,Sun 14 Aug 2005 10:17:57 +0800,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,1.0,ccert.edu.cn,163.con,sun,10,0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...
3,=?GB2312?B?tPq/qreixrE=?= <pan@12.com>,ling@ccert.edu.cn,Sun 14 Aug 2005 10:19:02 +0800,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,1.0,ccert.edu.cn,12.com,sun,10,0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...


In [21]:
## 特征工程四 ==> 邮件长度对是否是垃圾邮件的影响
def precess_content_length(lg):
    if lg <= 10:
        return 0
    elif lg <= 100:
        return 1
    elif lg <= 500:
        return 2
    elif lg <= 1000:
        return 3
    elif lg <= 1500:
        return 4
    elif lg <= 2000:
        return 5
    elif lg <= 2500:
        return 6
    elif lg <=  3000:
        return 7
    elif lg <= 4000:
        return 8
    elif lg <= 5000:
        return 9
    elif lg <= 10000:
        return 10
    elif lg <= 20000:
        return 11
    elif lg <= 30000:
        return 12
    elif lg <= 50000:
        return 13
    else:
        return 14

df['content_length'] = pd.Series(map(lambda st: len(st), df['content']))
df['content_length_type'] = pd.Series(map(lambda st: precess_content_length(st), df['content_length']))
df2 = df.groupby(['content_length_type', 'label'])['label'].agg(['count']).reset_index()
df3 = df2[df2.label == 1][['content_length_type', 'count']].rename(columns={'count':'c1'})
df4 = df2[df2.label == 0][['content_length_type', 'count']].rename(columns={'count':'c2'})
df5 = pd.merge(df3, df4)
df5['c1_rage'] = df5.apply(lambda r: r['c1'] / (r['c1'] + r['c2']), axis=1)
df5['c2_rage'] = df5.apply(lambda r: r['c2'] / (r['c1'] + r['c2']), axis=1)


print(df5.head(10))
# 画图
plt.plot(df5['content_length_type'], df5['c1_rage'], label=u'垃圾邮件比例')
plt.plot(df5['content_length_type'], df5['c2_rage'], label=u'正常邮件比例')
plt.grid(True)
plt.legend(loc = 0)
plt.show()

KeyError: 'content'

In [16]:
## 特征工程四 ==> 添加信号量
def process_content_sema(x):
    if x > 10000:
        return 0.5 / np.exp(np.log10(x) - np.log10(500)) + np.log(abs(x - 500) + 1) - np.log(abs(x - 10000)) + 1
    else:
        return 0.5 / np.exp(np.log10(x) - np.log10(500)) + np.log(abs(x - 500) + 1) + 1

a = np.arange(1,20000)
plt.plot(a, list(map(lambda t: process_content_sema(t), a)), label=u'信息量')
plt.grid(True)
plt.legend(loc = 0)
plt.show()

df['content_sema'] = list(map(lambda st: process_content_sema(st), df['content_length']))
df.head(2)

,from,to,date,content,label,to_address,from_address,date_week,date_hour,date_time_quantum,has_date,jieba_cut_content,content_length,content_length_type,content_sema
0,yan<(8月27-28上海)培训课程>,lu@ccert.edu.cn,Tue 30 Aug 2005 10:08:15 +0800,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课 程 背 景]每一...,1.0,ccert.edu.cn,unknown,tue,10,0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,1798,5,8.456151
1,pan <pan@jdl.ac.cn>,shi@ccert.edu.cn,Sun 14 Aug 2005 10:16:47 +0800,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,0.0,ccert.edu.cn,jdl.ac.cn,sun,10,0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,193,2,7.486084


In [17]:
## 查看列名称
df.dtypes

from                    object
to                      object
date                    object
content                 object
label                  float64
to_address              object
from_address            object
date_week               object
date_hour               object
date_time_quantum       object
has_date                 int64
jieba_cut_content       object
content_length           int64
content_length_type      int64
content_sema           float64
dtype: object

In [18]:
# 获取需要的列
df.drop(['from', 'to', 'date', 'content', 'to_address', 
         'from_address', 'date_week', 'date_hour', 'date_time_quantum', 
         'content_length', 'content_length_type'], 1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64620 entries, 0 to 64619
Data columns (total 4 columns):
label                64619 non-null float64
has_date             64620 non-null int64
jieba_cut_content    64620 non-null object
content_sema         64620 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.0+ MB


In [19]:
df.head(5)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409


In [20]:
# 结果输出csv文件
df.to_csv("../data/result_process02", encoding='utf-8', index=False)